<a href="https://colab.research.google.com/github/LUMII-AILab/NLP_Course/blob/main/notebooks/MSP/speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runas atpazīšana

In [ ]:
# Pārbauda GPU pieejamību un statusu
!nvidia-smi

# Instalē nepieciešamās izpildlaika bibliotēkas
!pip install transformers[torch] yt-dlp gradio jiwer openai-whisper diff-match-patch

In [2]:
from transformers import pipeline
from IPython.display import Audio, display, HTML
import jiwer
import whisper
from whisper.tokenizer import get_tokenizer
from diff_match_patch import diff_match_patch
import re
from itertools import zip_longest
import gradio as gr

In [ ]:
# Audio paraugs - Pasaules diktāta latviešu valodā lejupielāde no GitGub
!wget https://github.com/LUMII-AILab/NLP_Course/raw/refs/heads/main/notebooks/resources/ASR/diktats2024.mp3

# Audio paraugs - LTV ziņu fragmenta lejupielāde no YouTube
!yt-dlp -x --audio-format mp3 --download-sections "*00:00:00-00:00:31" -o "zinas1.%(ext)s" "https://www.youtube.com/watch?v=D5jMv2zBmbc"
# Ja tiek izmestas kļūdas "HTTP error 403 Forbidden" un "ffmpeg exited with code 1",
# izvēlieties citu YouTube video, atbilstoši precizējot lejuplādējamo laika intervālu

In [ ]:
# Audio paraugu priekšskatīšana
display(Audio('diktats2024.mp3'))
display(Audio('zinas1.mp3'))

In [11]:
# Palīgfunkcijas transkripciju apstrādei, sastatīšanai

def normalize(t):
    t = t.lower()
    t = re.sub(r'\W', ' ', t)
    t = re.sub(r'\s+', ' ', t)
    t = t.strip()
    return t

# Rakstzīmju līmeņa diff-match-patch:
# ievietošana un dzēšana, lai pred(icted) sastatītu ar ref(erence)
def show_aligment(pred, ref):
    dmp = diff_match_patch()
    diffs = dmp.diff_main(pred, ref)
    dmp.diff_cleanupSemantic(diffs)

    css = """
      <style>
        .d del{background:#ffe6e6;color:#7a1c1c;text-decoration:none;}
        .d ins{background:#e6ffed;color:#14532d;text-decoration:none;}
      </style>
    """
    esc = lambda s: s.replace("&","&amp;").replace("<","&lt;").replace(">","&gt;")

    out = []
    for op, data in diffs:
        s = esc(data)
        if op == 0:   out.append(s)
        if op == -1:  out.append(f"<del>{s}</del>")
        if op == 1:   out.append(f"<ins>{s}</ins>")

    display(HTML(css + "<div class='d'>" + "".join(out) + "</div>"))

def show_wer_alignment(w):
  import html as _html
  boxes = []

  for a in w.alignments[0]:
      hyp_slice = w.hypotheses[0][a.hyp_start_idx:a.hyp_end_idx]
      ref_slice = w.references[0][a.ref_start_idx:a.ref_end_idx]

      for hyp_tok, ref_tok in zip_longest(hyp_slice, ref_slice, fillvalue=''):
          hyp = _html.escape(hyp_tok or '∅')
          ref = _html.escape(ref_tok or '∅')
          t = a.type

          boxes.append(f"""
            <div class="align-box align-{t}">
              <div class="hyp">{hyp}</div>
              <div class="ref">{ref if t != 'equal' else '&nbsp;'}</div>
            </div>
          """)

  html = """
    <style>
      # .align-wrap { font-family: monospace; }
      .align-box {
        display: inline-block;
        border: 1px solid #eee;
      }
      .align-insert     { background: #89ff7d; }
      .align-delete     { background: #ff9898; }
      .align-substitute { background: #fffaf0; }
    </style>
    <div class="align-wrap">
    """ + "\n".join(boxes) + "</div>"

  display(HTML(html))

def evaluate(ref, pred, normalizer=normalize):
    if normalizer:
        ref = normalizer(ref)
        pred = normalizer(pred)

    w = jiwer.process_words(ref, pred)
    c = jiwer.process_characters(ref, pred)

    print('WER', w.wer, "hits", w.hits, "subs", w.substitutions, "dels", w.deletions, "ins", w.insertions, "n_ref", sum(len(x) for x in w.references), "n_hyp", sum(len(x) for x in w.hypotheses))
    print('CER', c.cer, "hits", c.hits, "subs", c.substitutions, "dels", c.deletions, "ins", c.insertions, "n_ref", sum(len(x) for x in c.references), "n_hyp", sum(len(x) for x in c.hypotheses))

    show_wer_alignment(w)
    show_aligment(ref, pred)

## Oriģinālie Whisper modeļi

| Size   | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|--------|------------|--------------------|---------------------|---------------|----------------|
| tiny   | 39 M       | tiny.en            | tiny                | ~1 GB         | ~10x           |
| base   | 74 M       | base.en            | base                | ~1 GB         | ~7x            |
| small  | 244 M      | small.en           | small               | ~2 GB         | ~4x            |
| medium | 769 M      | medium.en          | medium              | ~5 GB         | ~2x            |
| large  | 1550 M     | N/A                | large               | ~10 GB        | 1x             |
| turbo  | 809 M      | N/A                | turbo               | ~6 GB         | ~8x            |


In [ ]:
# Whisper lietošana no komandrindas
!whisper -h

In [12]:
# Whisper turbo modeļa izmantošana; automātiska valodas noteikšana
!whisper zinas1.mp3 --model turbo

100%|█████████████████████████████████████| 1.51G/1.51G [00:16<00:00, 98.8MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Latvian
[00:01.000 --> 00:07.820]  Pirms divām nedēļām negaidīti darbību apturēja Latvijā bāzētā aviokompānija SmartLinks Airlines.
[00:08.180 --> 00:15.940]  Līdzbiedrība, kuras nosaukuma iespējams daļa dzird pirmo reizi, vēl pirms gada tika atzīt par vērtīgāko pašmāju kompānija transporta nozerē.
[00:16.140 --> 00:22.140]  SmartLinks nodarbojās ar pilnu servisu lidmašīnu nomas un čarteru pakalpojamiem visā pasaulē.
[00:22.140 --> 00:30.880]  Kā minēts uzņēmuma mājas lapā, tam bijuši vairāk nekā 500 ofisa un tehniskie darbinieki, kā arī 1300 apkalpas locekļu.


In [13]:
# Mēģinājums ar sistēmas uzvedni uzlabot uzņēmuma nosaukuma atpazīšanu
!whisper zinas1.mp3 --model turbo --initial_prompt "SmartLynx"

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Latvian
[00:00.000 --> 00:07.840]  Pirms divām nedēļām negaidīti darbību apturēja Latvijā bāzētā aviokompānija SmartLynx Airlines.
[00:08.180 --> 00:15.940]  Līdzbiedrība, kuras nosaukuma iespējams daļa dzird pirmo reizi, vēl pirms gada tika atzīt par vērtīgāko pašmāju kompānija transporta nozerē.
[00:16.140 --> 00:22.140]  SmartLynx nodarbojās ar pilnu servisu lidmašīnu nomas un čarteru pakalpojamiem visā pasaulē.
[00:22.140 --> 00:30.880]  Kā minēts uzņēmuma mājas lapā, tam bijuši vairāk nekā 500 ofisa un tehniskie darbinieki, kā arī 1300 apkalpas locekļu.


In [14]:
# Salīdzinājums ar Whisper tiny modeli
!whisper zinas1.mp3 --model tiny

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 173MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Latvian
[00:00.000 --> 00:08.000]  Pirums divamnēdēļām negājēdīti darbīj baturēlāt vija bazātā avio kompānīs, smartling serlāns.
[00:08.000 --> 00:11.000]  Līdz biedrību kurus no saku minas pēns daļa dzīb pirmu arīs.
[00:11.000 --> 00:16.000]  Elpums gadatīkā dzīt par vērtīgā, ko pašmāj, kompānīt transportu nozere.
[00:16.000 --> 00:22.000]  Smartlings, noderboja, aserpīl, no servis, līdmas šīnu nuomas un charter, pakāl pojimiem visā pasālēt.
[00:22.000 --> 00:31.000]  Kā minēc uzņām mūma maislāpā, tam biji usšīvairāk nekāpiet, sīm tofīs saun techniskie darbīnīja, ka arī tūkstauds trīs sīmk abkāp sluocekļu.


In [15]:
# Turbo modelis vienmēr atgriež rezultātu oriģinālvalodā, pat ja norādīts "--task translate" parametrs.
# Lai izmantotu Whisper spējas ģenerēt transkripciju angliski, jāizmanto large vai medium modelis.
!whisper zinas1.mp3 --model large-v3 --language lv --task translate

100%|██████████████████████████████████████| 2.88G/2.88G [00:16<00:00, 182MiB/s]
[00:00.000 --> 00:08.000]  Two weeks ago, the airline Smartlinks Air Lines, based in Latvia, did not wait for work.
[00:08.000 --> 00:12.000]  The airline, which has been named for the first time for the first time,
[00:12.000 --> 00:16.000]  was named as the most valuable company of transport in the country.
[00:16.000 --> 00:22.000]  Smartlinks worked with full-service airline services and charter services all over the world.
[00:22.000 --> 00:31.000]  As mentioned, the company had more than 500 office and technical staff, as well as 1,300 store windows.


In [16]:
# Whisper lietošana Python kodā
model = whisper.load_model("turbo")

In [17]:
# Ielasa un priekšapstrādā audio paraugu
audio = whisper.load_audio("zinas1.mp3")
audio = whisper.pad_or_trim(audio)

# Sagatavo log-Mel spektrogrammu un pārvieto to uz procesoru, kur ielādēts modelis
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

# Nosaka runāto valodu
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# Dekodē runas datus
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

tokenizer = get_tokenizer(True)
print('Decoded', [tokenizer.encoding.decode([t]) for t in result.tokens])

# Izdrukā transkripciju
print(result)
print(result.text)

Detected language: lv
Decoded ['<|1.02|>', ' Pir', 'ms', ' div', 'ām', ' ned', 'ē', '�', '�', 'ām', ' neg', 'aid', 'ī', 'ti', ' dar', 'b', 'ī', 'bu', ' apt', 'ur', 'ē', 'ja', ' Lat', 'v', 'ij', 'ā', ' b', 'ā', 'z', 'ē', 't', 'ā', ' av', 'i', 'ok', 'omp', 'ān', 'ija', ' Smart', ' Links', ' Airlines', '.', '<|7.84|>', '<|8.20|>', ' L', 'ī', 'dz', 'b', 'ied', 'r', 'ī', 'ba', ',', ' kur', 'as', ' nos', 'au', 'k', 'umi', ' ', 'ies', 'p', 'ē', 'j', 'ams', ' da', '�', '�', 'a', ' dz', 'ird', ' pir', 'mo', ' re', 'izi', ',', ' v', 'ē', 'l', ' pir', 'ms', ' g', 'ada', ' t', 'ika', ' at', 'z', 'ī', 't', ' par', ' v', 'ē', 'rt', 'ī', 'g', 'ā', 'ko', ' pa', 'š', 'm', 'ā', 'ju', ' kom', 'p', 'ān', 'ija', ' transport', 'a', ' no', 'zer', 'ē', '.', '<|15.96|>', '<|16.16|>', ' Smart', ' Links', ' nod', 'ar', 'bo', 'j', 'ā', 's', ' ar', ' pil', 'nu', ' serv', 'isu', ' lid', 'ma', 'š', 'ī', 'nu', ' nom', 'as', ' un', ' č', 'arter', 'u', ' pak', 'al', 'po', 'jam', 'iem', ' vis', 'ā', ' pasa', 'ul', 'ē', 

## Hugging Face modeļu lietošana, novērtēšana

In [18]:
ref = """Aiva Kanepone
Putni brīnās par Liepāju
Kurš liepājnieks gan nezina, ka Liepājā, Zirgu salā, ir putnu vērošanas tornis? Bet kurš ir pamanījis, ka arī putni vēro Liepāju? Brīnās, bola acis un groza galvas, un bezkaunīgi planē pāri kafejnīcu galdiņiem, lai ieskatītos nelidojošo divkāju šķīvjos. Liepājā ir daudz noslēpumaina, par ko brīnīties. Piemēram, naktīs, kad ne tikvien mēness piesprausts pie debesu samta kā zelta ananass, bet arī zvaigznes izbārstītas no Dieva atvilktnēm. Tad tur, lejā, zīriņiem par brīnumu, kaut kas iekvēlojas kā milzīgs ugunskurs, un tam viducī vītero kanārijputniņu bars. „Kas tas ir? Kas tas ir?” sakliedzas kaijas. Tām neviens nav izstāstījis par koncertzāli „Lielais dzintars” un simfonisko orķestri. Vēl viņi nesaprot, kāpēc jaukie Liepājas baznīcu torņi ik pa laikam sajūk prātā un pēkšņi sākas tāda dārdoņa, it kā tajos būtu ielauzušies kazuāri: „Bom! Bom! Bom!”
Un kas notiek Graudu ielas smalkāko namu augstumos? Tur nometas estēti baloži, kas dievina jūgendstilu. Ak, tie lieliskie izciļņi, kur dūdojot griezt mīlas dejas, un iedobumi, kur sastiept liepās salauztos zariņus minimālisma stilā ieturētajām ligzdām!
Kormorāni, viltnieki, jau trešo gadu perina bruņotu laupīšanas uzbrukumu Zivju tirgum Liepājas promenādē, tikai nevar sagaidīt, kad to atkal atvērs. Čurkstes slepeni skatās Liepājas teātra izrādes, bet pēc tam sajūsmā savidžinās. Bet zivju gārņi sapņo, garās kājas cilājot, pastaigāt pa Rožu laukuma dobēm. Diemžēl gārņi kautrējas. Toties vārnai Liepājā ir piemineklis. Karr!
Visskaistākajos vakaros, spārnus plivinādami, putni plecu pie pleca salaižas uz Ziemeļu mola vērot saulrietu. Kormorāns blakus paugurknābja zosij, cekulpīle blakus purva ķauķim, un !pašā galā tārtiņš, no Līvu krasta atlaidies, klusiņām purpina lībiešu valodā.
Rudenī viņi lūr pa Liepājas skolu logiem. Ieraugot gramatiskās kļūdas, saķer apaļās galveles un spalgi čiepst priekšā: „Stulbs nav stūlbs, albums nav albūms, mobilais nav nekāds mobīlais! Čiv!”
Klau, vai tagad arī uz palodzes negrabinās viena zilzīlīte? Ko, ko viņa tur čiepst?
"""

### Novērtējam openai/whisper-tiny

In [ ]:
model = pipeline(task="automatic-speech-recognition", model="openai/whisper-tiny")

In [20]:
pred = model('diktats2024.mp3', return_timestamps=True)
pred['text']

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


' Ajvākā neponē, potnī brīnās par liapājā. Kūs liapānieks gan nezdīnākā liapājā, zergu salā ir potnumerušanas tuvarnis. Bet kūs ir pamaneis, ka arī potnīt veru liapājā. Brīnās, būl, atsis, un grūs, galvas, un beskā unīgi planējā parikā feinīt, un galdiniem, lai eskatīt uz neliduojošu, djūkāju, štīvijos. Liepājā ir daudz, noslēpā mēn parku, brīnīties. Piemēram naktis, kad ne tikvien menes piesprosts pie, dabas asam pkā zaldā ananas, bet arī zvajīgsnesis bārstītas no dīja vātvilk nēm. Tattur, lēja, zīrņiem per brīnu kautkas iekvēlu eskā milzīgs ugunskurs. Un tam vidu cīvītoru a kanāri putnīņu bārstas. Kas tas ir, kas tas ir saklīdzis kājez. Tam nevienst navistāstējis per konserdzāli lielēs dinters un symfonis ko Ortestri. Vēl viņi nesaprūt kapēc jau kēlības, baznīt sturni, ikpalēkam sa jūk prata, un peksņu sakas tāda darduņa. Tajos būtu, ila uz uzšies, ka zlāri, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, bāmu, b

In [21]:
evaluate(ref, pred['text'])

WER 1.0436241610738255 hits 49 subs 245 dels 4 ins 62 n_ref 298 n_hyp 356
CER 0.6471774193548387 hits 1335 subs 565 dels 84 ins 635 n_ref 1984 n_hyp 2535


### Novērtējam openai/whisper-large-v3-turbo

In [ ]:
model = pipeline(task="automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

In [23]:
pred = model('diktats2024.mp3', return_timestamps=True)
pred['text']

' Aiva Kanepone putni brīnās par liepāju. Kurš liepājnieks gan nezina, ka liepājā zirgu salā ir putnu vērošanas tornis, bet kurš ir pamanījis, ka arī putni vēro liepāju. Brīnās bola acis un groza galvas Un bezkaunīgi planē pāri kafeinīcu galdiņiem Lai ieskatītos nelidojošo divkāju šķīvjos Liepājā ir daudz noslēpumainu par ko brīnīties Piemēram naktīs, kad ne tik vien mēnesis Piesprausts pie debesu samta kā zelta ananas bet arī zvaigznes izbārstītas no dieva atvilknēm. Tad tur, lejā, zīriņiem par brīnumu, kaut kas iekvēlojas kā milzīgs ugunskurs un tam viducī vītero kanārī putniņu bars. Kas tas ir? Kas tas ir? sakliedzis kā ejas. Tām neviens nav izstāstījis par koncertzāli lielais dzintars un simfonisko orķestri Vēl viņi nesaprot, kāpēc jaukie liepājas baznītes tūrņi ik pa laikam sajūk prātā Un pēkšņi sākas tāda dārdoņa, it kā tajos būtu ielauzušies kazuāri BOM! BOM! BOM! Un kas notiek graudu ielas smalkāko namu augstumos? Tur nometas estēti baloži, kas dievina jūgenstil. Ak, tie lielis

In [24]:
evaluate(ref, pred['text'])

WER 0.13087248322147652 hits 263 subs 33 dels 2 ins 4 n_ref 298 n_hyp 300
CER 0.025705645161290324 hits 1945 subs 28 dels 11 ins 12 n_ref 1984 n_hyp 1985


### Novērtējam facebook/mms-1b-all

In [ ]:
model = pipeline(task="automatic-speech-recognition", model="facebook/mms-1b-all")

In [26]:
pred = model('diktats2024.mp3', generate_kwargs={"language": "lav", "task": "transcribe"}, chunk_length_s=30, stride_length_s=5)
pred['text']

'aiva kanepone putni brins par liepju kur liepajnieks gan nezina ka liepaja zirgu sala ir putnu vroanas tornis bet kurs ir pamanijis ka ari putni vero liepaju brins bola acis un groza galvas un bezkaunigi plane pri kafeinicu galdiniem lai ieskatitos nelidojoo diukaju stivjos liepja ir daudz noslepumaini par ko brinties piemram naktis kad ne tikvien menests piesprausts pie debesa samta ka zelta ananass bet ari zvaigznes izbarstitas no dieva atvilknem tad tur leja ziriniem par brinumu kaut kas iekvelojas ka milzigs ugunskurs un tam viducivitero kanri putninu bars kas tas ir kas tas ir sakliedzas kailas tam neviens nav izststijis par koncertzali lielais dzinters un simfonisko ortestri vel vini nesaprot kapec jaukie liepajas baznicas turni ikpalaikam sajuk prata un peknis sakas tada dardona itk tajos butu ielauzuies kazuari bom bo bo un kas notiek graudu ielast smalkko namu augstumos tur nometas estti baloi kas dievina jugenstil ak tie lieliskie izcilni kur dudojot griezt milas dejas un ie

In [27]:
evaluate(ref, pred['text'])

WER 0.5335570469798657 hits 140 subs 153 dels 5 ins 1 n_ref 298 n_hyp 294
CER 0.10887096774193548 hits 1774 subs 157 dels 53 ins 6 n_ref 1984 n_hyp 1937


## ASR demo saskarne

In [ ]:
model = pipeline(task="automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

In [29]:
def transcribe(audio):
    pred = model(audio, return_timestamps=True)
    print(pred)
    return pred.get("text", str(result))

gr.Interface(
    title="ASR demo",
    fn=transcribe,
    inputs=gr.Audio(sources=["microphone", "upload"], type="filepath"),
    outputs=gr.Textbox(),
).launch(share=True, debug=True)

# Lai veiktu ierakstīšanu no mikrofona, pārlūkā jāatver saite, kas šī koda izpildes laikā tiks uzģenerēta.
# Pārbaudīts ar Chrome (jāatļauj izmantot mikrofonu).
# Audio faila augšupielāde un transkribēšana var tikt darbināta tepat Colab vidē.

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://306cd49c4d55ca1b84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.


{'text': ' Vienas, divi, trīs, četri, pieci.', 'chunks': [{'timestamp': (0.0, 5.4), 'text': ' Vienas, divi, trīs, četri, pieci.'}]}
{'text': ' Aiva Kanepone putni brīnās par liepāju. Kurš liepājnieks gan nezina, ka liepājā zirgu salā ir putnu vērošanas tornis, bet kurš ir pamanījis, ka arī putni vēro liepāju. Brīnās bola acis un groza galvas Un bezkaunīgi planē pāri kafeinīcu galdiņiem Lai ieskatītos nelidojošo divkāju šķīvjos Liepājā ir daudz noslēpumainu par ko brīnīties Piemēram naktīs, kad ne tik vien mēnesis Piesprausts pie debesu samta kā zelta ananas bet arī zvaigznes izbārstītas no dieva atvilknēm. Tad tur, lejā, zīriņiem par brīnumu, kaut kas iekvēlojas kā milzīgs ugunskurs un tam viducī vītero kanārī putniņu bars. Kas tas ir? Kas tas ir? sakliedzis kā ejas. Tām neviens nav izstāstījis par koncertzāli lielais dzintars un simfonisko orķestri Vēl viņi nesaprot, kāpēc jaukie liepājas baznītes tūrņi ik pa laikam sajūk prātā Un pēkšņi sākas tāda dārdoņa, it kā tajos būtu ielauzušie